In [ ]:
import sys
  
# append the path of the parent directory
sys.path.append("../")

from src import imageUtils as img
import tesserocr
from PIL import Image
import re

In [ ]:
#### CONSTANTS #######
pathToVideo = r'C:\Users\hkuser\Videos\2023-03-17 09-25-05.mkv'
outputPath = 'output'

In [ ]:
frames = img.getFramesFromVideo(pathToVideo=pathToVideo, outputPath=outputPath, shouldOutputToFile=True)

In [ ]:
height, width, channels = frames[1].shape
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class Item:
    box:tuple #left upper right lower
    name:str
    sides:list
    numbersOnly:bool
    

BOUNDING_BOXES = {
    'IN_GAME' :
    {
        'Time': Item((875,0,1025,65), 'Time', ['none'], False),
        'TeamName': Item((185,85,400,125),'TeamName', ['left','right'], False),
        'Score': Item((700,0,825,85),'Score', ['left', 'right'], True)    
    }
}


NO_REVERSE_BOX = {'Score'}

def parseToActualValue(key:str, val:str):
    if key == 'Time':
        return parseTimeDuration(val)
    elif key in {'Score'}:
        return int(val)
    return val

def parseTimeDuration(timeStr:str) -> timedelta:
    minutes,seconds = timeStr.split(':')[0], timeStr.split(':')[1]
    if seconds[0] == '9':
        seconds = '2' + seconds[1]
    return timedelta(minutes=int(minutes), seconds=int(seconds))

def imageTransformation(im:Image, type:str) -> list:
    res = []
    if type.startswith('Score'):
        hsv = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2HSV)
        #lower = np.array([0, 0, 215])
        #upper = np.array([179, 61, 255])
        lower = np.array([10,0,218])
        upper = np.array([179,39,255])
        mask = cv2.inRange(hsv, lower, upper)
        im1 = Image.fromarray(255-mask)
        res.append(im1)
        
        #alternative
        lower = np.array([0,0,196])
        upper = np.array([179,39,255])
        mask = cv2.inRange(hsv, lower, upper)
        im2 = Image.fromarray(255-mask)
        res.append(im2)
    elif type.startswith('Time'):
         hsv = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2HSV)
         im = Image.fromarray(hsv)
         res.append(im)
    else:
        print('No transformation found')
    return res

def getSubImageBox(item:Item, side:str) -> tuple:
    validSides = {'left', 'right', 'none'}
    if side not in validSides:
        raise ValueError(f'side must be {validSides}')
    
    box = item.box
    if side == 'right':
        box = (width - box[2], box[1], width - box[0], box[3])
    return box

def parseImage(im:Image, numbersOnly:bool=False) -> str:
    with tesserocr.PyTessBaseAPI() as api:
        api.SetPageSegMode(tesserocr.PSM.SINGLE_BLOCK)
        if numbersOnly:
            api.SetVariable("tessedit_char_whitelist", "0123456789")
        api.SetImage(im)
        res = api.GetUTF8Text()
        return re.sub(r'\n', '', res)

def getAllItemsFromImage(im:Image, debug:bool=False) -> dict:
    res = {}
    skipMoment = False
    for moment,boxes in BOUNDING_BOXES.items():
        skipMoment=False
        for key,item in boxes.items():
            for side in item.sides:
                imgs = imageTransformation(im.crop(getSubImageBox(item, side)),f'{key}/{side}')
                for im in imgs:
                    imgVal = im
                    parsedVal = parseImage(imgVal)
                    try:
                        parsedVal = parseToActualValue(key,parsedVal)
                    except Exception as err:
                        if debug:
                            print(f'Exception when parasing {key}. Text=`{parsedVal}`')
                            pass
                        else:
                            print(f'Was not {moment} because {key} could not be parsed. Text=`{parsedVal}`')
                            print(f'{err}')
                            skipMoment=True
                            continue
                    skipMoment=False
                    res[f'{key}/{side}'] = {'image': imgVal, 'value': parsedVal}
                if skipMoment:
                    break
            if skipMoment:
                break
    return res

import numpy as np


def getResultsPerFrame(frames:list) -> list:
    return [getAllItemsFromImage(Image.fromarray(im)) for im in frames]

In [ ]:
    Image.fromarray(ogimg).crop(getSubImageBox('Score', 'left'))


In [ ]:
import cv2
import numpy as np

def nothing(x):
    pass

# Load image
image = frames[76]

# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
# Hue is from 0-179 for Opencv
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

while(1):
    # Get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Convert to HSV format and color threshold
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display result image
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
ogimg = cv2.colorChange(frames[76], cv2.BG

im = Image.fromarray(ogimg)
im

In [ ]:
#ogimg = frames[1] #cv2.cvtColor(frames[1], cv2.COLOR_BGR2GRAY)
#ogimg= cv2.adaptiveThreshold(cv2.GaussianBlur( (5,5),2),255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                         cv2.THRESH_BINARY, 155, 40)

#ogimg = cv2.cvtColor(frames[1], cv2.COLOR_BGR2GRAY)

# Otsu's thresholding

ogimg = frames[76]

im = Image.fromarray(ogimg)
items = getAllItemsFromImage(im, debug=True)
for key,val in items.items():
    display(key)
    display(val['image'])
    display(val['value'])

In [ ]:
res = getResultsPerFrame(frames)
res

In [ ]:
for i in range(len(res)):
    if len(res[i])==0:
        display(f'Frame {i}')
        display(Image.fromarray(frames[i]))


In [ ]:
print(tesserocr.tesseract_version())  # print tesseract-ocr version
print(tesserocr.get_languages())  # prints tessdata path and list of available languages